<a href="https://colab.research.google.com/github/Martoto/BigData_ForecastHackathon/blob/main/silicionauta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%cd Users/daniel.sant.salles/

/mnt/batch/tasks/shared/LS_root/mounts/clusters/memoryman/code/Users/daniel.sant.salles


In [1]:
%pip install pandas pyarrow seaborn plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests

file_path = '/content/hackathon_2025_templates.zip'
file_id = '1Ed-nqUyCT0z-T0AvHwLMsGqHYqjw2248' # This is the file ID

if not os.path.exists(file_path):
    print(f'{file_path} not found. Downloading...')

    # Construct the direct download URL
    download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

    try:
        response = requests.get(download_url, stream=True)
        response.raise_for_status() # Raise an exception for bad status codes

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f'{file_path} downloaded successfully.')

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
        print("Please ensure the file is shared publicly or with 'Anyone with the link'.")

else:
    print(f'{file_path} found.')

/content/hackathon_2025_templates.zip not found. Downloading...


FileNotFoundError: [Errno 2] No such file or directory: '/content/hackathon_2025_templates.zip'

In [8]:
import zipfile
import os

zip_path = 'hackathon_2025_templates.zip'
extract_dir = './data/'

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f'Extracted {zip_path} to {extract_dir}')
else:
    print(f'{zip_path} not found in the current directory.')

Extracted hackathon_2025_templates.zip to ./data/


In [7]:
import pandas as pd
file_path = './data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
df_pdv = pd.read_parquet(file_path)
display(df_pdv.head())

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [5]:
import pandas as pd
file_path = './data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet'
df_transactions = pd.read_parquet(file_path)
display(df_transactions.head())

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


In [11]:
import pandas as pd
import pyarrow.parquet as pq

# Get unique product IDs from transactions for filtering
valid_product_ids = set(df_transactions['internal_product_id'].unique())
print(f"Found {len(valid_product_ids):,} unique product IDs in transactions")

file_path = './data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet'
table = pq.ParquetFile(file_path)

# Collect filtered chunks
filtered_chunks = []
total_rows_processed = 0
total_rows_kept = 0

for batch in table.iter_batches(batch_size=5000):
    df_chunk = batch.to_pandas()
    total_rows_processed += len(df_chunk)
    
    # Filter to keep only products that exist in transactions
    filtered_chunk = df_chunk[df_chunk['produto'].isin(valid_product_ids)]
    total_rows_kept += len(filtered_chunk)
    
    if len(filtered_chunk) > 0:
        filtered_chunks.append(filtered_chunk)

# Combine all filtered chunks
df_products = pd.concat(filtered_chunks, ignore_index=True) if filtered_chunks else pd.DataFrame()

print(f"Total rows processed: {total_rows_processed:,}")
print(f"Total rows kept: {total_rows_kept:,}")
print(f"Filtering efficiency: {(total_rows_kept/total_rows_processed)*100:.2f}%")
print(f"Final products dataframe shape: {df_products.shape}")
print("\nFirst few rows of filtered products:")
display(df_products.head())

Found 7,092 unique product IDs in transactions
Total rows processed: 7,092
Total rows kept: 7,092
Filtering efficiency: 100.00%
Final products dataframe shape: (7092, 8)

First few rows of filtered products:


,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,2282334733936076502,Distilled Spirits,JOSEPH CARTRON CAFÉ LIQUEUR,Distilled Spirits,Core,Liqueurs & Cordials,Joseph Cartron Cafe,Spiribam
1,6091840953834683482,Distilled Spirits,SPRINGBANK 18 YEAR SINGLE MALT 700ML,Distilled Spirits,Specialty,Scotch Whisky,Springbank 18 Year Single Malt,Pacific Edge Wine & Spirits
2,1968645851245092408,Distilled Spirits,J BRANDT TRIPLE SEC 12/750ML 30PF,Distilled Spirits,Private Label,Liqueurs & Cordials,J Brandt Triple Sec,Sazerac Spirits
3,994706710729219179,Draft,REFORMATION CASHMERE IPA 1/4 KEG,Draft,In&Out,Other Draft,Reformation Cashmere Fresh Hop IPA,Reformation Brewery
4,9209550539540384349,Non-Alcohol,HELLA MOSCOW MULE 750ML,Non Alcohol,Core,Mixers,Hella Bitters Bloody Mary,Hella Bitter Llc


In [12]:
# Verification of the filtering results
print("🔍 FILTERING VERIFICATION")
print("="*50)

# Check unique products in filtered dataset
unique_products_filtered = df_products['produto'].nunique()
print(f"Unique products in filtered dataset: {unique_products_filtered:,}")

# Check if all transaction products are covered
transaction_products = set(df_transactions['internal_product_id'].unique())
filtered_products = set(df_products['produto'].unique())

print(f"Unique products in transactions: {len(transaction_products):,}")
print(f"Unique products in filtered products: {len(filtered_products):,}")

# Check intersection
intersection = transaction_products.intersection(filtered_products)
print(f"Products in both datasets: {len(intersection):,}")

# Check if there are products in transactions not in products
missing_in_products = transaction_products - filtered_products
if missing_in_products:
    print(f"⚠️  Products in transactions but missing in products: {len(missing_in_products)}")
    print(f"Examples: {list(missing_in_products)[:10]}")
else:
    print("✅ All transaction products are covered in the products dataset")

# Check if there are products in products not in transactions
missing_in_transactions = filtered_products - transaction_products
if missing_in_transactions:
    print(f"⚠️  Products in products but missing in transactions: {len(missing_in_transactions)}")
    print(f"Examples: {list(missing_in_transactions)[:10]}")
else:
    print("✅ All products have corresponding transactions")

print(f"\n📊 Data consistency: {'Perfect match!' if len(missing_in_products) == 0 and len(missing_in_transactions) == 0 else 'Some mismatches found'}")

🔍 FILTERING VERIFICATION
Unique products in filtered dataset: 7,092
Unique products in transactions: 7,092
Unique products in filtered products: 7,092
Products in both datasets: 7,092
✅ All transaction products are covered in the products dataset
✅ All products have corresponding transactions

📊 Data consistency: Perfect match!


1- pdv	premise	categoria_pdv	zipcode 
2- internal_store_id	internal_product_id	distributor_id	transaction_date	reference_date	quantity	gross_value	net_value	gross_profit	discount	taxes
3- produto categoria descricao tipos label subcategoria marca fabricante               


## Visão Geral da Estrutura dos Dados

### 📊 Dataset 1: Ponto de Venda (PDV)
| Coluna | Descrição |
|--------|-----------|
| `pdv` | Identificador do Ponto de Venda |
| `premise` | Informações da loja/estabelecimento |
| `categoria_pdv` | Classificação da categoria do PDV |
| `zipcode` | Código postal/CEP |

### 💳 Dataset 2: Transações
| Coluna | Descrição |
|--------|-----------|
| `internal_store_id` | Identificador interno da loja |
| `internal_product_id` | Identificador interno do produto |
| `distributor_id` | Identificador do distribuidor |
| `transaction_date` | Data da transação |
| `reference_date` | Data de referência |
| `quantity` | Quantidade da transação |
| `gross_value` | Valor bruto da transação |
| `net_value` | Valor líquido da transação |
| `gross_profit` | Lucro bruto |
| `discount` | Desconto aplicado |
| `taxes` | Valor dos impostos |

### 🛍️ Dataset 3: Produtos
| Coluna | Descrição |
|--------|-----------|
| `produto` | Identificador do produto |
| `categoria` | Categoria do produto |
| `descricao` | Descrição do produto |
| `tipos` | Tipos do produto |
| `label` | Rótulo do produto |
| `subcategoria` | Subcategoria do produto |
| `marca` | Marca |
| `fabricante` | Fabricante |


# 🚀 EXPLORATORY DATA ANALYSIS - HACKATHON 2025
## Going HAM on the Data! 💪

Let's dive deep into this retail dataset and uncover insights for forecasting!

In [5]:
# Import comprehensive analysis libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configure visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("🚀 Libraries loaded successfully! Let's go HAM on this data!")

🚀 Libraries loaded successfully! Let's go HAM on this data!


In [10]:
# 📊 LOAD ALL DATASETS
print("🔄 Loading all three datasets...")

# Dataset 1: PDV (Points of Sale)
pdv_file = './data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
df_pdv = pd.read_parquet(pdv_file)
print(f"✅ PDV Dataset loaded: {df_pdv.shape}")

# Dataset 2: Transactions
transactions_file = './data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet'
df_transactions = pd.read_parquet(transactions_file)
print(f"✅ Transactions Dataset loaded: {df_transactions.shape}")

# Dataset 3: Products (using chunked reading for large file)
products_file = './data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet'
df_products = pd.read_parquet(products_file)
print(f"✅ Products Dataset loaded: {df_products.shape}")

print(f"\n🎯 Total datasets loaded: 3")
print(f"📈 Combined data points: {df_pdv.shape[0] + df_transactions.shape[0] + df_products.shape[0]:,}")

🔄 Loading all three datasets...
✅ PDV Dataset loaded: (14419, 4)
✅ Transactions Dataset loaded: (6560698, 11)
✅ Products Dataset loaded: (7092, 8)

🎯 Total datasets loaded: 3
📈 Combined data points: 6,582,209


In [13]:
# 🔍 DATASET STRUCTURE ANALYSIS - Going Deep!
print("="*80)
print("🔍 DEEP DIVE INTO DATASET STRUCTURES")
print("="*80)

datasets = {
    'PDV': df_pdv,
    'Transactions': df_transactions,
    'Products': df_products
}

for name, df in datasets.items():
    print(f"\n📊 {name.upper()} DATASET ANALYSIS:")
    print(f"   📏 Shape: {df.shape}")
    print(f"   📋 Columns: {list(df.columns)}")
    print(f"   🏷️  Data Types:")
    for col, dtype in df.dtypes.items():
        print(f"      {col}: {dtype}")
    print(f"   💾 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   🕳️  Missing Values: {df.isnull().sum().sum()}")
    print(f"   🔢 Unique Values per Column:")
    for col in df.columns:
        unique_count = df[col].nunique()
        print(f"      {col}: {unique_count:,}")
    print("-" * 60)

🔍 DEEP DIVE INTO DATASET STRUCTURES

📊 PDV DATASET ANALYSIS:
   📏 Shape: (14419, 4)
   📋 Columns: ['pdv', 'premise', 'categoria_pdv', 'zipcode']
   🏷️  Data Types:
      pdv: object
      premise: object
      categoria_pdv: object
      zipcode: int32
   💾 Memory Usage: 2.62 MB
   🕳️  Missing Values: 0
   🔢 Unique Values per Column:
      pdv: 14,419
      premise: 2
      categoria_pdv: 54
      zipcode: 788
------------------------------------------------------------

📊 TRANSACTIONS DATASET ANALYSIS:
   📏 Shape: (6560698, 11)
   📋 Columns: ['internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']
   🏷️  Data Types:
      internal_store_id: object
      internal_product_id: object
      distributor_id: object
      transaction_date: object
      reference_date: object
      quantity: float64
      gross_value: float64
      net_value: float64
      gross_profit: flo

In [ ]:
# 🧹 DATA QUALITY ASSESSMENT - The Deep Clean!
print("="*80)
print("🧹 DATA QUALITY ASSESSMENT")
print("="*80)

def analyze_data_quality(df, name):
    print(f"\n🔍 {name} QUALITY REPORT:")
    
    # Missing values analysis
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    
    print(f"📊 Missing Values Analysis:")
    for col in df.columns:
        if missing_values[col] > 0:
            print(f"   {col}: {missing_values[col]:,} ({missing_percent[col]:.2f}%)")
    
    # Duplicates analysis
    duplicates = df.duplicated().sum()
    print(f"🔄 Duplicate Rows: {duplicates:,} ({(duplicates/len(df)*100):.2f}%)")
    
    # Numerical columns analysis
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    if len(numerical_cols) > 0:
        print(f"📈 Numerical Columns Analysis:")
        for col in numerical_cols:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            outliers = ((df[col] < (q1 - 1.5 * iqr)) | (df[col] > (q3 + 1.5 * iqr))).sum()
            print(f"   {col}: Min={df[col].min():.2f}, Max={df[col].max():.2f}, Outliers={outliers:,}")
    
    print("-" * 60)

# Analyze each dataset
for name, df in datasets.items():
    analyze_data_quality(df, name)

In [14]:
# 💰 SALES TRANSACTION ANALYSIS - The Money Maker!
print("="*80)
print("💰 SALES TRANSACTION ANALYSIS")
print("="*80)

# Convert date columns to datetime
if 'transaction_date' in df_transactions.columns:
    df_transactions['transaction_date'] = pd.to_datetime(df_transactions['transaction_date'])
if 'reference_date' in df_transactions.columns:
    df_transactions['reference_date'] = pd.to_datetime(df_transactions['reference_date'])

# Time period analysis
print(f"📅 Transaction Date Range:")
if 'transaction_date' in df_transactions.columns:
    print(f"   From: {df_transactions['transaction_date'].min()}")
    print(f"   To: {df_transactions['transaction_date'].max()}")
    print(f"   Duration: {(df_transactions['transaction_date'].max() - df_transactions['transaction_date'].min()).days} days")

# Financial metrics overview
numerical_cols = ['quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']
available_cols = [col for col in numerical_cols if col in df_transactions.columns]

print(f"\n💵 Financial Metrics Summary:")
if available_cols:
    for col in available_cols:
        print(f"   {col}:")
        print(f"      Total: ${df_transactions[col].sum():,.2f}")
        print(f"      Average: ${df_transactions[col].mean():.2f}")
        print(f"      Median: ${df_transactions[col].median():.2f}")
        print(f"      Std Dev: ${df_transactions[col].std():.2f}")

# Top performing stores and products
print(f"\n🏪 Top 10 Stores by Total Revenue:")
if 'internal_store_id' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    top_stores = df_transactions.groupby('internal_store_id')['gross_value'].sum().sort_values(ascending=False).head(10)
    for store_id, revenue in top_stores.items():
        print(f"   Store {store_id}: ${revenue:,.2f}")

print(f"\n🛍️ Top 10 Products by Total Sales:")
if 'internal_product_id' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    top_products = df_transactions.groupby('internal_product_id')['gross_value'].sum().sort_values(ascending=False).head(10)
    for product_id, revenue in top_products.items():
        print(f"   Product {product_id}: ${revenue:,.2f}")

💰 SALES TRANSACTION ANALYSIS
📅 Transaction Date Range:
   From: 2022-01-01 00:00:00
   To: 2022-12-31 00:00:00
   Duration: 364 days

💵 Financial Metrics Summary:
   quantity:
      Total: $53,311,532.07
      Average: $8.13
      Median: $2.00
      Std Dev: $80.49
   gross_value:
      Total: $805,333,770.76
      Average: $122.75
      Median: $42.10
      Std Dev: $866.43
   net_value:
      Total: $781,531,966.87
      Average: $119.12
      Median: $40.77
      Std Dev: $865.18
   gross_profit:
      Total: $143,882,148.80
      Average: $21.93
      Median: $10.51
      Std Dev: $232.65
   discount:
      Total: $181,513,749.61
      Average: $27.67
      Median: $2.30
      Std Dev: $384.76
   taxes:
      Total: $23,801,803.14
      Average: $3.63
      Median: $0.54
      Std Dev: $11.31

🏪 Top 10 Stores by Total Revenue:
   Store 6491855528940268514: $2,932,767.13
   Store 3025867614395044464: $2,684,152.80
   Store 4374038751643985193: $2,616,642.68
   Store 719590676618757

In [15]:
# 📈 TIME SERIES ANALYSIS - The Time Machine!
print("="*80)
print("📈 TIME SERIES ANALYSIS")
print("="*80)

if 'transaction_date' in df_transactions.columns:
    # Extract time components
    df_transactions['year'] = df_transactions['transaction_date'].dt.year
    df_transactions['month'] = df_transactions['transaction_date'].dt.month
    df_transactions['day'] = df_transactions['transaction_date'].dt.day
    df_transactions['weekday'] = df_transactions['transaction_date'].dt.day_name()
    df_transactions['week'] = df_transactions['transaction_date'].dt.isocalendar().week
    
    # Daily sales analysis
    daily_sales = df_transactions.groupby('transaction_date').agg({
        'gross_value': ['sum', 'mean', 'count'],
        'quantity': 'sum'
    }).round(2)
    
    print(f"📊 Daily Sales Statistics:")
    print(f"   Total Days with Transactions: {len(daily_sales)}")
    print(f"   Average Daily Revenue: ${daily_sales[('gross_value', 'sum')].mean():,.2f}")
    print(f"   Best Day Revenue: ${daily_sales[('gross_value', 'sum')].max():,.2f}")
    print(f"   Worst Day Revenue: ${daily_sales[('gross_value', 'sum')].min():,.2f}")
    
    # Weekly patterns
    weekly_patterns = df_transactions.groupby('weekday')['gross_value'].sum().sort_values(ascending=False)
    print(f"\n📅 Weekly Sales Patterns:")
    for day, revenue in weekly_patterns.items():
        print(f"   {day}: ${revenue:,.2f}")
    
    # Monthly trends
    if 'gross_value' in df_transactions.columns:
        monthly_trends = df_transactions.groupby(['year', 'month'])['gross_value'].sum()
        print(f"\n🗓️ Monthly Revenue Trends:")
        for (year, month), revenue in monthly_trends.items():
            print(f"   {year}-{month:02d}: ${revenue:,.2f}")

# Seasonality detection
print(f"\n🌟 Seasonality Insights:")
if 'month' in df_transactions.columns:
    seasonal_pattern = df_transactions.groupby('month')['gross_value'].mean().round(2)
    peak_month = seasonal_pattern.idxmax()
    low_month = seasonal_pattern.idxmin()
    print(f"   Peak Sales Month: {peak_month} (${seasonal_pattern[peak_month]:,.2f} avg)")
    print(f"   Lowest Sales Month: {low_month} (${seasonal_pattern[low_month]:,.2f} avg)")

📈 TIME SERIES ANALYSIS
📊 Daily Sales Statistics:
   Total Days with Transactions: 365
   Average Daily Revenue: $2,206,393.89
   Best Day Revenue: $269,828,419.09
   Worst Day Revenue: $11,394.54

📅 Weekly Sales Patterns:
   Sunday: $280,198,190.63
   Monday: $129,634,657.15
   Tuesday: $124,884,143.82
   Wednesday: $108,631,312.81
   Thursday: $91,117,453.89
   Friday: $68,646,346.69
   Saturday: $2,221,665.77

🗓️ Monthly Revenue Trends:
   2022-01: $31,089,589.20
   2022-02: $34,985,233.80
   2022-03: $42,767,181.34
   2022-04: $39,708,678.19
   2022-05: $48,529,123.84
   2022-06: $51,197,744.23
   2022-07: $41,707,373.43
   2022-08: $51,961,556.34
   2022-09: $312,855,647.57
   2022-10: $47,909,054.74
   2022-11: $50,067,765.72
   2022-12: $52,554,822.35

🌟 Seasonality Insights:
   Peak Sales Month: 9 ($378.86 avg)
   Lowest Sales Month: 1 ($80.98 avg)


In [16]:
# 🛍️ PRODUCT PORTFOLIO ANALYSIS - The Product Universe!
print("="*80)
print("🛍️ PRODUCT PORTFOLIO ANALYSIS")
print("="*80)

# Product categories analysis
print("📦 Product Categories Overview:")
if 'categoria' in df_products.columns:
    category_counts = df_products['categoria'].value_counts()
    print(f"   Total Categories: {len(category_counts)}")
    print("   Top 10 Categories:")
    for cat, count in category_counts.head(10).items():
        print(f"      {cat}: {count:,} products")

# Brand analysis
print(f"\n🏷️ Brand Analysis:")
if 'marca' in df_products.columns:
    brand_counts = df_products['marca'].value_counts()
    print(f"   Total Brands: {len(brand_counts)}")
    print("   Top 10 Brands:")
    for brand, count in brand_counts.head(10).items():
        print(f"      {brand}: {count:,} products")

# Manufacturer analysis
print(f"\n🏭 Manufacturer Analysis:")
if 'fabricante' in df_products.columns:
    mfg_counts = df_products['fabricante'].value_counts()
    print(f"   Total Manufacturers: {len(mfg_counts)}")
    print("   Top 10 Manufacturers:")
    for mfg, count in mfg_counts.head(10).items():
        print(f"      {mfg}: {count:,} products")

# Product types analysis
print(f"\n🎯 Product Types Distribution:")
if 'tipos' in df_products.columns:
    type_counts = df_products['tipos'].value_counts()
    print(f"   Total Types: {len(type_counts)}")
    for ptype, count in type_counts.head(15).items():
        print(f"      {ptype}: {count:,} products")

🛍️ PRODUCT PORTFOLIO ANALYSIS
📦 Product Categories Overview:
   Total Categories: 7
   Top 10 Categories:
      Distilled Spirits: 2,202 products
      Wine: 1,879 products
      Package: 1,403 products
      Draft: 918 products
      Non-Alcohol: 597 products
      ABA Spirits: 91 products
      Tobacco: 2 products

🏷️ Brand Analysis:
   Total Brands: 4221
   Top 10 Brands:
      Barrell Bourbon- Private Label: 60 products
      Bud Light: 38 products
      Budweiser: 36 products
      Barrell Stellum - Private Label: 35 products
      Fireball Cinnamon Whiskey: 28 products
      Buffalo Trace Kentucky Straight Bourbon Whiskey: 27 products
      Yukon Jack Canadian Whiskey Honey: 24 products
      Stella Artois: 21 products
      Michelob Ultra: 21 products
      Chi-Chi's Variety Pack: 20 products

🏭 Manufacturer Analysis:
   Total Manufacturers: 343
   Top 10 Manufacturers:
      Sazerac Spirits: 1,169 products
      AB Anheuser Busch Inc: 878 products
      Tilray Brands: 311 produ

In [17]:
# 🗺️ GEOGRAPHICAL ANALYSIS - The Map Masters!
print("="*80)
print("🗺️ GEOGRAPHICAL ANALYSIS")
print("="*80)

# PDV location analysis
if 'zipcode' in df_pdv.columns:
    zipcode_analysis = df_pdv['zipcode'].value_counts()
    print(f"📍 Zipcode Distribution:")
    print(f"   Total Unique Zipcodes: {len(zipcode_analysis)}")
    print("   Top 15 Zipcodes by Store Count:")
    for zip_code, count in zipcode_analysis.head(15).items():
        print(f"      {zip_code}: {count:,} stores")

# PDV category analysis
if 'categoria_pdv' in df_pdv.columns:
    pdv_categories = df_pdv['categoria_pdv'].value_counts()
    print(f"\n🏪 PDV Categories:")
    print(f"   Total Categories: {len(pdv_categories)}")
    for category, count in pdv_categories.items():
        print(f"      {category}: {count:,} stores")

# Store premise analysis
if 'premise' in df_pdv.columns:
    premise_analysis = df_pdv['premise'].value_counts()
    print(f"\n🏢 Store Premise Types:")
    print(f"   Total Premise Types: {len(premise_analysis)}")
    for premise, count in premise_analysis.head(10).items():
        print(f"      {premise}: {count:,} stores")

print("-" * 60)

🗺️ GEOGRAPHICAL ANALYSIS
📍 Zipcode Distribution:
   Total Unique Zipcodes: 788
   Top 15 Zipcodes by Store Count:
      80202: 230 stores
      30096: 133 stores
      80205: 129 stores
      80211: 127 stores
      81301: 123 stores
      80524: 113 stores
      80501: 108 stores
      30120: 105 stores
      30161: 105 stores
      80424: 103 stores
      80903: 103 stores
      30721: 102 stores
      80203: 102 stores
      80525: 100 stores
      80302: 100 stores

🏪 PDV Categories:
   Total Categories: 54
      Restaurant: 3,316 stores
      Convenience: 2,849 stores
      Package/Liquor: 2,153 stores
      Bar: 1,444 stores
      Grocery: 697 stores
      Mexican Rest: 623 stores
      Other On Premise: 473 stores
      Hotel/Motel: 359 stores
      Asian: 263 stores
      Pizza: 187 stores
      Super Center: 181 stores
      Golf - Public: 168 stores
      Drug: 167 stores
      Italian: 144 stores
      Service Org: 138 stores
      Special Event: 137 stores
      Golf - Priv

In [18]:
# 📈 ADVANCED VISUALIZATIONS - The Eye Candy!
print("="*80)
print("📈 CREATING KILLER VISUALIZATIONS")
print("="*80)

# Create a subplot figure for comprehensive analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Daily Sales Trend', 'Weekly Pattern', 'Top Categories', 'Revenue Distribution'),
    specs=[[{"secondary_y": True}, {"type": "bar"}],
           [{"type": "pie"}, {"type": "histogram"}]]
)

# 1. Daily sales trend
if 'transaction_date' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    daily_sales = df_transactions.groupby('transaction_date')['gross_value'].sum().reset_index()
    fig.add_trace(
        go.Scatter(x=daily_sales['transaction_date'], y=daily_sales['gross_value'],
                  mode='lines+markers', name='Daily Sales'),
        row=1, col=1
    )

# 2. Weekly patterns
if 'weekday' in df_transactions.columns:
    weekly_data = df_transactions.groupby('weekday')['gross_value'].sum().reset_index()
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly_data['weekday'] = pd.Categorical(weekly_data['weekday'], categories=weekday_order, ordered=True)
    weekly_data = weekly_data.sort_values('weekday')
    
    fig.add_trace(
        go.Bar(x=weekly_data['weekday'], y=weekly_data['gross_value'],
               name='Weekly Sales', marker_color='lightblue'),
        row=1, col=2
    )

# 3. Top product categories (if we can link products to transactions)
if 'categoria' in df_products.columns:
    top_categories = df_products['categoria'].value_counts().head(8)
    fig.add_trace(
        go.Pie(labels=top_categories.index, values=top_categories.values,
               name="Categories"),
        row=2, col=1
    )

# 4. Revenue distribution
if 'gross_value' in df_transactions.columns:
    fig.add_trace(
        go.Histogram(x=df_transactions['gross_value'], name='Revenue Distribution',
                    nbinsx=50, marker_color='orange'),
        row=2, col=2
    )

# Update layout
fig.update_layout(height=800, showlegend=True, 
                  title_text="🚀 Comprehensive Retail Analytics Dashboard")
fig.show()

print("✅ Interactive dashboard created!")

📈 CREATING KILLER VISUALIZATIONS


NameError: name 'make_subplots' is not defined

In [19]:
# 🔗 CORRELATION ANALYSIS - Finding Hidden Relationships!
print("="*80)
print("🔗 CORRELATION ANALYSIS")
print("="*80)

# Financial metrics correlation
financial_cols = ['quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']
available_financial = [col for col in financial_cols if col in df_transactions.columns]

if len(available_financial) > 1:
    correlation_matrix = df_transactions[available_financial].corr()
    
    # Create correlation heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, linewidths=0.5, cbar_kws={"shrink": .8})
    plt.title('💰 Financial Metrics Correlation Matrix', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("🔍 Key Correlations:")
    # Find strongest correlations
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if abs(corr_val) > 0.7:  # Strong correlation
                col1, col2 = correlation_matrix.columns[i], correlation_matrix.columns[j]
                print(f"   {col1} ↔ {col2}: {corr_val:.3f}")

# Distributor performance analysis
if 'distributor_id' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    distributor_performance = df_transactions.groupby('distributor_id').agg({
        'gross_value': ['sum', 'mean', 'count'],
        'quantity': 'sum',
        'gross_profit': 'sum' if 'gross_profit' in df_transactions.columns else 'mean'
    }).round(2)
    
    print(f"\n🚚 Top 10 Distributors by Revenue:")
    top_distributors = df_transactions.groupby('distributor_id')['gross_value'].sum().sort_values(ascending=False).head(10)
    for dist_id, revenue in top_distributors.items():
        transactions_count = df_transactions[df_transactions['distributor_id'] == dist_id].shape[0]
        print(f"   Distributor {dist_id}: ${revenue:,.2f} ({transactions_count:,} transactions)")

print("-" * 60)

🔗 CORRELATION ANALYSIS


NameError: name 'plt' is not defined

In [24]:

# --- VERIFICAÇÃO ---
# Vamos verificar se os arquivos foram carregados corretamente.

print("--- 1. DADOS DAS LOJAS (PDV) ---")
print(f"Shape: {df_pdv.shape}") # Mostra (linhas, colunas)
display(df_pdv.head())         # Mostra as 5 primeiras linhas
print("\\n" + "="*50 + "\\n")

print("--- 2. DADOS DAS TRANSAÇÕES ---")
print(f"Shape: {df_transactions.shape}")
display(df_transactions.head())
print("\\n" + "="*50 + "\\n")

print("--- 3. DADOS DOS PRODUTOS ---")
print(f"Shape: {df_products.shape}")
display(df_products.head())

--- 1. DADOS DAS LOJAS (PDV) ---
Shape: (14419, 4)


,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


\n==================================================\n
--- 2. DADOS DAS TRANSAÇÕES ---
Shape: (6560698, 16)


,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,year,month,day,weekday,week
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,2022,7,13,Wednesday,28
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,2022,3,21,Monday,12
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,2022,9,6,Tuesday,36
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,2022,9,11,Sunday,36
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,2022,2,18,Friday,7


\n==================================================\n
--- 3. DADOS DOS PRODUTOS ---
Shape: (7092, 8)


,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,2282334733936076502,Distilled Spirits,JOSEPH CARTRON CAFÉ LIQUEUR,Distilled Spirits,Core,Liqueurs & Cordials,Joseph Cartron Cafe,Spiribam
1,6091840953834683482,Distilled Spirits,SPRINGBANK 18 YEAR SINGLE MALT 700ML,Distilled Spirits,Specialty,Scotch Whisky,Springbank 18 Year Single Malt,Pacific Edge Wine & Spirits
2,1968645851245092408,Distilled Spirits,J BRANDT TRIPLE SEC 12/750ML 30PF,Distilled Spirits,Private Label,Liqueurs & Cordials,J Brandt Triple Sec,Sazerac Spirits
3,994706710729219179,Draft,REFORMATION CASHMERE IPA 1/4 KEG,Draft,In&Out,Other Draft,Reformation Cashmere Fresh Hop IPA,Reformation Brewery
4,9209550539540384349,Non-Alcohol,HELLA MOSCOW MULE 750ML,Non Alcohol,Core,Mixers,Hella Bitters Bloody Mary,Hella Bitter Llc


In [25]:
# --- PASSO 1: JUNTAR OS DATAFRAMES (MERGE) ---

print("Iniciando a junção das tabelas...")

# 1. Juntamos as transações com os dados dos produtos
# A conexão é feita usando 'internal_product_id' e 'produto' como chaves.
df_merged = pd.merge(
    df_transactions, 
    df_products,
    left_on='internal_product_id',
    right_on='produto',
    how='left'  # 'how=left' garante que todas as transações sejam mantidas.
)

print(f"Após juntar com produtos, a tabela tem {df_merged.shape[1]} colunas.")


# 2. Agora, juntamos o resultado anterior com os dados das lojas (PDV)
# A conexão é feita usando 'internal_store_id' e 'pdv' como chaves.
df_merged = pd.merge(
    df_merged,
    df_pdv,
    left_on='internal_store_id',
    right_on='pdv',
    how='left'
)

print(f"Após juntar com as lojas, a tabela final tem {df_merged.shape[1]} colunas.")


# --- VERIFICAÇÃO ---
print("\\nJunção concluída! Vamos verificar o resultado:")

# O número de linhas deve ser o mesmo da tabela de transações original
print(f"Shape da tabela original de transações: {df_transactions.shape}")
print(f"Shape da tabela final combinada:      {df_merged.shape}")

# Verifique que a tabela agora tem colunas das 3 tabelas originais
print("\\nExemplo da tabela combinada (note as colunas 'marca', 'categoria', 'premise', etc.):")
display(df_merged.head())

Iniciando a junção das tabelas...
Após juntar com produtos, a tabela tem 24 colunas.
Após juntar com as lojas, a tabela final tem 28 colunas.
\nJunção concluída! Vamos verificar o resultado:
Shape da tabela original de transações: (6560698, 16)
Shape da tabela final combinada:      (6560698, 28)
\nExemplo da tabela combinada (note as colunas 'marca', 'categoria', 'premise', etc.):


,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,...,descricao,tipos,label,subcategoria,marca,fabricante,pdv,premise,categoria_pdv,zipcode
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,...,BUD LIGHT CHELADA FUEGO 15/25 CN,Package,Core,Specialty,Bud Light Chelada Fuego,AB Anheuser Busch Inc,7384367747233276219,Off Premise,Package/Liquor,80905.0
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,...,MICHELOB ULTRA 18/12 CN,Package,Core,Lager,Michelob Ultra,AB Anheuser Busch Inc,3536908514005606262,Off Premise,Package/Liquor,80239.0
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,...,BUD LIGHT LIME 18/12 CN,Package,Core,Lager,Bud Light Lime,AB Anheuser Busch Inc,3138231730993449825,Off Premise,Package/Liquor,80634.0
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,...,99 BUTTERSCOTCH 12/10/50ML 99PF,Allocated Spirits,None,Liqueurs & Cordials,99 Butterscotch,Sazerac Spirits,3681167389484217654,Off Premise,Package/Liquor,80226.0
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,...,NB VOODOO RANGER IMPERIAL IPA 15/19.2 CN,Package,Core,IPA,New Belgium Voodoo Ranger Imperial IPA,NB New Belgium,7762413312337359369,Off Premise,Convenience,30096.0


In [29]:
# Vamos verificar o tipo de dado (dtype) da coluna transaction_date
print("Tipo de dado da coluna 'transaction_date':")
print(df_merged['transaction_date'].dtype)

Tipo de dado da coluna 'transaction_date':
datetime64[ns]


In [26]:
# --- PASSO DE DIAGNÓSTICO: INVESTIGAR A SAÚDE DOS DADOS ---

print("Iniciando diagnóstico do dataframe 'df_merged'...")
print("="*50 + "\\n")


# --- Diagnóstico 1: Valores Nulos (Buracos nos dados) ---
# Vamos contar quantos valores nulos (NaN) existem em cada coluna.
print("--- 1. Contagem de Valores Nulos por Coluna ---")
missing_values = df_merged.isnull().sum()
print(missing_values[missing_values > 0]) # Mostra apenas as colunas que TÊM valores nulos
print("\\n")


# --- Diagnóstico 2: Tipos de Dados ---
# Vamos verificar se o pandas entendeu cada coluna corretamente (ex: data é data, número é número).
print("--- 2. Informações Gerais e Tipos de Dados ---")
# O .info() nos dá um resumo completo, incluindo o tipo (Dtype) e a contagem de valores não-nulos.
df_merged.info()
print("\\n")


# --- Diagnóstico 3: Estatísticas de Colunas Numéricas ---
# Vamos olhar um resumo estatístico das colunas com números.
# Isso ajuda a encontrar coisas estranhas, como valores negativos ou absurdamente altos.
print("--- 3. Resumo Estatístico das Colunas Numéricas ---")
# O .describe() mostra contagem, média, desvio padrão, mínimo, máximo, etc.
display(df_merged[['quantity', 'gross_value', 'net_value', 'gross_profit']].describe())


Iniciando diagnóstico do dataframe 'df_merged'...
==================================================\n
--- 1. Contagem de Valores Nulos por Coluna ---
label            526953
subcategoria      10312
pdv               45582
premise           45582
categoria_pdv     45582
zipcode           45582
dtype: int64
\n
--- 2. Informações Gerais e Tipos de Dados ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6560698 entries, 0 to 6560697
Data columns (total 28 columns):
 #   Column               Dtype         
---  ------               -----         
 0   internal_store_id    object        
 1   internal_product_id  object        
 2   distributor_id       object        
 3   transaction_date     datetime64[ns]
 4   reference_date       datetime64[ns]
 5   quantity             float64       
 6   gross_value          float64       
 7   net_value            float64       
 8   gross_profit         float64       
 9   discount             float64       
 10  taxes                float64    

,quantity,gross_value,net_value,gross_profit
count,6.560698e+06,6.560698e+06,6.560698e+06,6.560698e+06
mean,8.125893e+00,1.227512e+02,1.191233e+02,2.193092e+01
std,8.049387e+01,8.664265e+02,8.651757e+02,2.326470e+02
min,-1.530000e+03,-4.267290e+04,-3.984800e+04,-2.743960e+05
25%,1.000000e+00,2.803500e+01,2.711032e+01,7.140242e+00
50%,2.000000e+00,4.210000e+01,4.077048e+01,1.051024e+01
75%,4.000000e+00,9.200000e+01,8.787900e+01,2.173200e+01
max,9.423000e+04,6.041739e+05,6.041739e+05,2.744160e+05


In [27]:
# --- DIAGNÓSTICO MAIS PRECISO ---

print("Iniciando diagnóstico preciso do dataframe 'df_merged'...")
print("="*50 + "\\n")


# --- Diagnóstico 4: Análise de Linhas Duplicadas ---
# Linhas 100% idênticas podem indicar erros de registro e inflar os dados.
print("--- 4. Verificação de Linhas Duplicadas ---")
num_duplicatas = df_merged.duplicated().sum()
print(f"Número de linhas completamente duplicadas encontradas: {num_duplicatas:,}")
if num_duplicatas > 0:
    # Mostra um exemplo das linhas que são duplicadas
    print("\\nExemplo de linhas duplicadas:")
    display(df_merged[df_merged.duplicated(keep=False)].sort_values(by=list(df_merged.columns)).head(10))
print("\\n")


# --- Diagnóstico 5: Análise de Conteúdo das Colunas de Texto ---
# O comando .value_counts() é excelente para isso. Ele conta quantas vezes cada valor aparece.
# O `dropna=False` nos ajuda a contar os valores nulos (NaN) também.

print("--- 5. Análise de Conteúdo (Valores Mais Comuns) ---")

print("\\n--- Categoria do Produto ('categoria') ---")
print(df_merged['categoria'].value_counts(dropna=False).head(10))
print("-" * 30)

print("\\n--- Tipo de Loja ('premise') ---")
print(df_merged['premise'].value_counts(dropna=False).head(10))
print("-" * 30)

print("\\n--- Categoria da Loja ('categoria_pdv') ---")
print(df_merged['categoria_pdv'].value_counts(dropna=False).head(15)) # Mostrando 15 para ter mais detalhes
print("-" * 30)

Iniciando diagnóstico preciso do dataframe 'df_merged'...
==================================================\n
--- 4. Verificação de Linhas Duplicadas ---
Número de linhas completamente duplicadas encontradas: 0
\n
--- 5. Análise de Conteúdo (Valores Mais Comuns) ---
\n--- Categoria do Produto ('categoria') ---
categoria
Package              4625080
Non-Alcohol           955097
Distilled Spirits     611285
Draft                 192300
Wine                  124232
ABA Spirits            52274
Tobacco                  430
Name: count, dtype: int64
------------------------------
\n--- Tipo de Loja ('premise') ---
premise
Off Premise    5823678
On Premise      691438
NaN              45582
Name: count, dtype: int64
------------------------------
\n--- Categoria da Loja ('categoria_pdv') ---
categoria_pdv
Convenience         2126380
Package/Liquor      2016477
Grocery             1276042
Super Center         365825
Restaurant           261861
Bar                  192643
NaN                 

In [28]:
# --- Análise Completa da Coluna 'categoria_pdv' ---

print("Vamos investigar todos os tipos de loja existentes.")

# 1. Contar quantos tipos únicos de loja (PDV) existem no total
num_categorias_loja = df_merged['categoria_pdv'].nunique()
print(f"No total, encontramos {num_categorias_loja} tipos diferentes de lojas.\\n")

# 2. Mostrar a contagem para CADA tipo de loja, ordenado do mais comum para o menos comum
print("Abaixo está a lista completa de todas as categorias de loja e o número de transações em cada uma:")
# O display() do jupyter notebook formata a saída de forma mais legível
display(df_merged['categoria_pdv'].value_counts(dropna=False))

Vamos investigar todos os tipos de loja existentes.
No total, encontramos 54 tipos diferentes de lojas.\n
Abaixo está a lista completa de todas as categorias de loja e o número de transações em cada uma:


categoria_pdv
Convenience                  2126380
Package/Liquor               2016477
Grocery                      1276042
Super Center                  365825
Restaurant                    261861
Bar                           192643
NaN                            45582
Mexican Rest                   43712
Hotel/Motel                    20827
Golf - Public                  19807
Drug                           19159
Service Org                    17230
Pizza                          14995
Billiard/Bowling               14239
Golf - Private                 13713
Sports/Rec Club                11223
Italian                         9659
Irish                           9213
Asian                           7922
Other On Premise                7026
Stadium/Concession              7021
Bodega                          5284
Barbeque                        4932
Club Store                      4798
Military                        4724
Church                          4696
Airline/Airport         

In [30]:
# --- PASSO 2: LIMPEZA (VERSÃO AVANÇADA COM REGRA PARA 'PREMISE') ---

print("Iniciando a limpeza com a regra customizada para 'premise'...")

df_clean = df_merged.copy()

# Ações preliminares que já tínhamos definido:
df_clean = df_clean[df_clean['quantity'] > 0]
df_clean = df_clean[df_clean['transaction_date'].dt.year == 2022]
print("1. Linhas com quantidade inválida e fora de 2022 removidas.")

# Preenche nulos nas outras colunas, mas DEIXA 'premise' em paz por enquanto
for col in ['categoria', 'marca', 'categoria_pdv']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('Unknown')
print("2. Nulos em outras colunas de texto preenchidos.")


# Ação principal: Implementar a regra para preencher 'premise'
print("3. Aplicando a regra para deduzir 'premise' a partir de 'categoria_pdv'...")

# Listas de mapeamento baseadas na nossa análise
on_premise_categorias = [
    'Restaurant', 'Bar', 'Mexican Rest', 'Hotel/Motel', 'Golf - Public', 'Pizza', 
    'Billiard/Bowling', 'Golf - Private', 'Sports/Rec Club', 'Italian', 'Irish', 
    'Asian', 'Stadium/Concession', 'Barbeque', 'German', 'Casino', 'Other On Premise'
]
off_premise_categorias = [
    'Convenience', 'Package/Liquor', 'Grocery', 'Super Center', 'Drug', 
    'Club Store', 'Bodega', 'Mass Merch'
]

# Função que contém a nossa lógica
def deduzir_premise(row):
    # Se 'premise' já tem um valor e não é nulo, mantém ele.
    if pd.notna(row['premise']):
        return row['premise']
    
    # Se 'premise' for nulo, tentamos deduzir...
    if row['categoria_pdv'] in on_premise_categorias:
        return 'On Premise'
    elif row['categoria_pdv'] in off_premise_categorias:
        return 'Off Premise'
    
    # Se a categoria da loja também for desconhecida ou não estiver no nosso mapa,
    # então não conseguimos deduzir.
    else:
        return 'Unknown'

# O comando .apply executa nossa função em cada linha do dataframe.
# O resultado será salvo na própria coluna 'premise'.
df_clean['premise'] = df_clean.apply(deduzir_premise, axis=1)
print("4. Lógica de dedução para 'premise' aplicada com sucesso!")


# --- VERIFICAÇÃO ---
print("\\nLimpeza e dedução de 'premise' concluídas!")

print("\\nContagem de valores na coluna 'premise' ANTES da nossa regra (no df_merged):")
print(df_merged['premise'].value_counts(dropna=False))

print("\\nContagem de valores na coluna 'premise' DEPOIS da nossa regra (no df_clean):")
print(df_clean['premise'].value_counts(dropna=False))


Iniciando a limpeza com a regra customizada para 'premise'...
1. Linhas com quantidade inválida e fora de 2022 removidas.
2. Nulos em outras colunas de texto preenchidos.
3. Aplicando a regra para deduzir 'premise' a partir de 'categoria_pdv'...
4. Lógica de dedução para 'premise' aplicada com sucesso!
\nLimpeza e dedução de 'premise' concluídas!
\nContagem de valores na coluna 'premise' ANTES da nossa regra (no df_merged):
premise
Off Premise    5823678
On Premise      691438
NaN              45582
Name: count, dtype: int64
\nContagem de valores na coluna 'premise' DEPOIS da nossa regra (no df_clean):
premise
Off Premise    5739410
On Premise      684470
Unknown           6281
Name: count, dtype: int64


In [38]:
# --- PASSO 3 (VERSÃO FINAL): CRIAR FEATURES DE ESTADO E CIDADE ---

print("Instalando/Verificando a biblioteca pgeocode...")
%pip install pgeocode -q

import pgeocode
import pandas as pd

print("Preparando para mapear CEPs para estado e cidade...")
nomi = pgeocode.Nominatim('us')
df_clean['zipcode_str'] = df_clean['zipcode'].astype(str).str.slice(0, 5)

print("Mapeando...")
zip_codes = df_clean['zipcode_str'].unique()
state_info = nomi.query_postal_code(zip_codes)

# Dicionário para mapear CEP -> Estado
zip_to_state = pd.Series(state_info.state_code.values, index=state_info.postal_code).to_dict()
# Dicionário para mapear CEP -> Cidade
zip_to_city = pd.Series(state_info.place_name.values, index=state_info.postal_code).to_dict()

# Mapear as novas colunas
df_clean['state'] = df_clean['zipcode_str'].map(zip_to_state)
df_clean['city'] = df_clean['zipcode_str'].map(zip_to_city)

# Preencher o que não foi encontrado
df_clean['state'].fillna('Unknown', inplace=True)
df_clean['city'].fillna('Unknown', inplace=True)

print("\\nMAPEAMENTO CONCLUÍDO!")
print("Colunas 'state' e 'city' criadas.")

Instalando/Verificando a biblioteca pgeocode...
Note: you may need to restart the kernel to use updated packages.
Preparando para mapear CEPs para estado e cidade...
Mapeando...


C:\Users\danie\AppData\Local\Temp\ipykernel_27404\3386771597.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['state'].fillna('Unknown', inplace=True)
C:\Users\danie\AppData\Local\Temp\ipykernel_27404\3386771597.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

\nMAPEAMENTO CONCLUÍDO!
Colunas 'state' e 'city' criadas.


In [37]:
# --- Investigando os CEPs não encontrados ---

print("Filtrando o dataframe para mostrar apenas as linhas onde o estado é 'Unknown'...")

# Cria um novo dataframe apenas com as linhas problemáticas
ceps_problematicos_df = df_clean[df_clean['state'] == 'Unknown']

print(f"Encontramos {len(ceps_problematicos_df['zipcode'].unique())} CEPs únicos que não foram mapeados.\\n")

print("Abaixo estão os valores únicos da coluna ORIGINAL 'zipcode' que falharam:")
# O .unique() nos mostra todos os valores diferentes que existem na coluna
display(ceps_problematicos_df['zipcode'].unique())

print("\\nE aqui estão esses mesmos valores depois da nossa limpeza para 5 dígitos ('zipcode_str'):")
display(ceps_problematicos_df['zipcode_str'].unique())

Filtrando o dataframe para mostrar apenas as linhas onde o estado é 'Unknown'...
Encontramos 21 CEPs únicos que não foram mapeados.\n
Abaixo estão os valores únicos da coluna ORIGINAL 'zipcode' que falharam:


array([   nan, 30136., 30221., 30211., 30155., 30640., 30245., 30202.,
       30201., 30020., 30190., 80039., 90920., 90132.,  8107., 80328.,
       80147., 31972., 81153., 80029., 90919.])

\nE aqui estão esses mesmos valores depois da nossa limpeza para 5 dígitos ('zipcode_str'):


array(['nan', '30136', '30221', '30211', '30155', '30640', '30245',
       '30202', '30201', '30020', '30190', '80039', '90920', '90132',
       '8107.', '80328', '80147', '31972', '81153', '80029', '90919'],
      dtype=object)

In [ ]:
# --- PASSO 3.1: CORREÇÃO MANUAL DE ESTADO E CIDADE ---
print("Aplicando correção manual para estado e cidade...")

correction_map = {
    '30221': {'state': 'GA', 'city': 'Grayson'},
    '30211': {'state': 'GA', 'city': 'Dacula'},
    '30245': {'state': 'GA', 'city': 'Lawrenceville'},
    '30020': {'state': 'GA', 'city': 'Clarkdale'},
    '90132': {'state': 'CA', 'city': 'Los Angeles'},
    '80328': {'state': 'CO', 'city': 'Boulder'},
    '81153': {'state': 'CO', 'city': 'San Pablo'}
}

def corrigir_local(row):
    if row['zipcode_str'] in correction_map:
        # Pega os dados do nosso mapa de correção
        info = correction_map[row['zipcode_str']]
        row['state'] = info.get('state', 'Unknown')
        row['city'] = info.get('city', 'Unknown')
    return row

df_clean = df_clean.apply(corrigir_local, axis=1)

print("Correção manual concluída!")
print("\\nContagem de 'Unknown' em 'state' após correção:")
print(df_clean['state'].value_counts().get('Unknown', 0))

In [39]:
# --- PASSO 3.1 (VERSÃO REVISADA E CORRIGIDA) ---

print("Aplicando correção manual para estado e cidade...")

# 1. Dicionário de correção (com a chave 'state' corrigida para 81153)
correction_map = {
    '30221': {'state': 'GA', 'city': 'Grayson'},
    '30211': {'state': 'GA', 'city': 'Dacula'},
    '30245': {'state': 'GA', 'city': 'Lawrenceville'},
    '30020': {'state': 'GA', 'city': 'Clarkdale'},
    '90132': {'state': 'CA', 'city': 'Los Angeles'},
    '80328': {'state': 'CO', 'city': 'Boulder'},
    '81153': {'state': 'CO', 'city': 'San Pablo'}  # Erro corrigido aqui
}

# 2. Contar 'Unknowns' ANTES da correção para podermos comparar
unknowns_antes = df_clean['state'].value_counts().get('Unknown', 0)
print(f"Número de 'Unknowns' antes da correção: {unknowns_antes}")

# 3. Iterar pelo mapa e aplicar as correções de forma segura usando .loc
# .loc é uma forma eficiente de selecionar e alterar dados específicos
for zip_code, info in correction_map.items():
    # Criamos uma "máscara" para encontrar as linhas exatas que queremos alterar:
    # Onde o CEP é o que queremos corrigir E o estado atual é 'Unknown'
    mask = (df_clean['zipcode_str'] == zip_code) & (df_clean['state'] == 'Unknown')
    
    # Usamos a máscara para atualizar 'state' e 'city' apenas nessas linhas
    df_clean.loc[mask, 'state'] = info['state']
    df_clean.loc[mask, 'city'] = info['city']

# --- VERIFICAÇÃO ---
print("\\nCorreção manual concluída!")
unknowns_depois = df_clean['state'].value_counts().get('Unknown', 0)
print(f"Número de 'Unknowns' DEPOIS da correção: {unknowns_depois}")

# Verificando se a cidade também foi preenchida
print("\\nExemplo de uma das correções feitas:")
display(df_clean[df_clean['zipcode_str'] == '30221'][['zipcode_str', 'state', 'city']].head(1))

Aplicando correção manual para estado e cidade...
Número de 'Unknowns' antes da correção: 24165
\nCorreção manual concluída!
Número de 'Unknowns' DEPOIS da correção: 17592
\nExemplo de uma das correções feitas:


,zipcode_str,state,city
3650,30221,GA,Grayson


In [40]:
# --- PASSO 4: AGREGAÇÃO SEMANAL COM CIDADE ---

print("Iniciando agregação de dados por semana (incluindo estado e cidade)...")

# Cria a coluna 'week' para podermos agrupar por ela
df_clean['week'] = df_clean['transaction_date'].dt.isocalendar().week

# Define todas as colunas que queremos manter após o agrupamento.
# Note que 'state' e 'city' estão na lista.
grouping_keys = [
    'week', 
    'internal_store_id', 
    'internal_product_id', 
    'categoria', 
    'marca', 
    'premise', 
    'categoria_pdv',
    'state',
    'city'
]

# Agrupa os dados usando as chaves definidas e soma a quantidade de vendas para cada grupo
weekly_data = df_clean.groupby(grouping_keys).agg(
    total_quantity=('quantity', 'sum')
).reset_index()


# --- VERIFICAÇÃO ---
print(f"\\nAgregação concluída! De {len(df_clean):,} transações, criamos {len(weekly_data):,} linhas de dados semanais.")
print("Cada linha agora representa o total vendido de um produto em uma loja/cidade em uma semana.")
display(weekly_data.head())

Iniciando agregação de dados por semana (incluindo estado e cidade)...
\nAgregação concluída! De 6,430,161 transações, criamos 6,140,206 linhas de dados semanais.
Cada linha agora representa o total vendido de um produto em uma loja/cidade em uma semana.


,week,internal_store_id,internal_product_id,categoria,marca,premise,categoria_pdv,state,city,total_quantity
0,1,1001371918471115422,1029370090212151375,Package,Michelob Ultra,Off Premise,Convenience,GA,Talking Rock,3.0
1,1,1001371918471115422,1120490062981954254,Package,Bud Light,Off Premise,Convenience,GA,Talking Rock,18.0
2,1,1001371918471115422,2239307647969388381,Package,Natural Light,Off Premise,Convenience,GA,Talking Rock,2.0
3,1,1001371918471115422,4353552881410365573,Package,Natural Light,Off Premise,Convenience,GA,Talking Rock,7.0
4,1,1001371918471115422,4797439216678436447,Package,Bud Light Lime,Off Premise,Convenience,GA,Talking Rock,1.0


In [41]:
# --- PASSO 5: CRIAÇÃO DE FEATURES DE SÉRIES TEMPORAIS ---

print("Criando features de lag e média móvel no 'weekly_data'...")

# 1. Ordenar os dados é crucial para que a sequência de semanas esteja correta para cada produto
weekly_data = weekly_data.sort_values(by=['internal_store_id', 'internal_product_id', 'week'])

# 2. Criar "Lag Features": a venda da semana passada, retrasada, etc.
# O groupby garante que o .shift() seja calculado para cada produto/loja separadamente, sem misturar os históricos.
for lag in [1, 2, 3, 4]:
    weekly_data[f'quantity_lag_{lag}'] = weekly_data.groupby(['internal_store_id', 'internal_product_id'])['total_quantity'].shift(lag)

# 3. Criar "Rolling Features": a média de vendas das últimas semanas.
# Isso ajuda a suavizar ruídos e capturar tendências. Usamos .shift(1) para não usar a venda da semana atual no cálculo da média do passado.
for window in [2, 4, 8]:
    weekly_data[f'quantity_rolling_avg_{window}'] = weekly_data.groupby(['internal_store_id', 'internal_product_id'])['total_quantity'].shift(1).rolling(window=window, min_periods=1).mean()

# 4. Preencher os valores nulos (NaN) que foram criados com zero.
# Isso acontece nas primeiras semanas do histórico de cada produto, onde não há dados passados para calcular lag/média.
weekly_data.fillna(0, inplace=True)


# --- VERIFICAÇÃO ---
print("\\nCriação de features concluída!")
print("Nosso dataset está 100% pronto para o treinamento do modelo.")
print("Note as novas colunas 'quantity_lag_...' e 'quantity_rolling_avg_...':")
display(weekly_data.head(10))

Criando features de lag e média móvel no 'weekly_data'...
\nCriação de features concluída!
Nosso dataset está 100% pronto para o treinamento do modelo.
Note as novas colunas 'quantity_lag_...' e 'quantity_rolling_avg_...':


,week,internal_store_id,internal_product_id,categoria,marca,premise,categoria_pdv,state,city,total_quantity,quantity_lag_1,quantity_lag_2,quantity_lag_3,quantity_lag_4,quantity_rolling_avg_2,quantity_rolling_avg_4,quantity_rolling_avg_8
433888,6,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
529002,7,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,2.0,1.0,0.0,0.0,0.0,1.0,1.000000,1.000000
2042472,21,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,1.0,2.0,1.0,0.0,0.0,1.5,1.500000,1.500000
2521528,25,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,2.0,1.0,2.0,1.0,0.0,1.5,1.333333,1.333333
2891177,28,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,2.0,2.0,1.0,2.0,1.0,1.5,1.500000,1.500000
3599754,34,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,2.0,2.0,2.0,1.0,2.0,2.0,1.750000,1.600000
4481600,39,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,1.0,2.0,2.0,2.0,1.0,2.0,1.750000,1.666667
5321037,46,1000237487041964405,1837429607327399565,Package,Fire Maker Perfect Match IPA,On Premise,Winery,GA,Talking Rock,2.0,1.0,2.0,2.0,2.0,1.5,1.750000,1.571429
1565795,17,1000237487041964405,4038588102284338370,Package,Jekyll Cooter Brown,On Premise,Winery,GA,Talking Rock,1.0,0.0,0.0,0.0,0.0,1.0,1.666667,1.571429
433889,6,1000237487041964405,5429216175252037173,Package,Jekyll Southern Juice IPA,On Premise,Winery,GA,Talking Rock,2.0,0.0,0.0,0.0,0.0,0.0,1.500000,1.666667


In [43]:
# --- LISTANDO ESTADOS E CIDADES PRESENTES NOS DADOS ---

print("Gerando a lista de todos os estados e cidades únicos no nosso conjunto de dados...")
print("="*70)

# 1. Contar e listar os estados únicos
# .unique() pega todos os valores diferentes e .tolist() transforma em uma lista python
estados_unicos = sorted(weekly_data['state'].unique())
print(f"\\nEncontramos {len(estados_unicos)} estados únicos em nossos dados:")
print(estados_unicos)
print("-" * 70)

# 2. Contar as cidades únicas
cidades_unicas = weekly_data['city'].unique()
print(f"\\nEncontramos {len(cidades_unicas)} cidades únicas.")
print("-" * 70)


# 3. Mostrar as cidades agrupadas por estado (para melhor organização)
print("\\nPara referência, aqui estão as cidades que temos em cada estado:")
cidades_por_estado = weekly_data.groupby('state')['city'].unique()
# Usamos display para uma formatação mais amigável no notebook
display(cidades_por_estado)

Gerando a lista de todos os estados e cidades únicos no nosso conjunto de dados...
\nEncontramos 4 estados únicos em nossos dados:
['CA', 'CO', 'GA', 'Unknown']
----------------------------------------------------------------------
\nEncontramos 500 cidades únicas.
----------------------------------------------------------------------
\nPara referência, aqui estão as cidades que temos em cada estado:


state
CA                                             [Los Angeles]
CO         [Grand Junction, Denver, Arvada, Longmont, Lit...
GA         [Talking Rock, Atlanta, Brunswick, Canton, Alp...
Unknown                                            [Unknown]
Name: city, dtype: object

In [44]:
# --- PASSO 6: CRIAR FEATURES DE FERIADO E INTERAÇÃO ---

print("Criando features de feriado com base na sua sugestão...")
import numpy as np

# 1. Lista das semanas de 2022 que contêm feriados importantes nos EUA
# (Ex: MLK Day, Memorial Day, 4th of July, Labor Day, Thanksgiving, Christmas)
holiday_weeks = [3, 8, 22, 25, 27, 36, 41, 45, 47, 52]

# 2. Criar uma feature binária simples para marcar se a semana tem feriado ou não
# Usamos .isin() para verificar se o valor da coluna 'week' está na nossa lista
weekly_data['is_holiday_week'] = weekly_data['week'].isin(holiday_weeks).astype(int)


# 3. Criar as "Features de Interação" que você idealizou
# Elas combinam a informação do feriado com o tipo de local (premise)
# O modelo agora pode aprender pesos diferentes para cada uma dessas condições

# Feature para feriado em locais On-Premise
weekly_data['holiday_on_premise'] = np.where(
    (weekly_data['is_holiday_week'] == 1) & (weekly_data['premise'] == 'On Premise'), 1, 0
)

# Feature para feriado em locais Off-Premise
weekly_data['holiday_off_premise'] = np.where(
    (weekly_data['is_holiday_week'] == 1) & (weekly_data['premise'] == 'Off Premise'), 1, 0
)


# --- VERIFICAÇÃO ---
print("\\nCriação das features de feriado concluída!")

# Vamos verificar a contagem de cada nova feature
print(f"Semanas marcadas como feriado: {weekly_data['is_holiday_week'].sum():,}")
print(f"Semanas de feriado em locais On Premise: {weekly_data['holiday_on_premise'].sum():,}")
print(f"Semanas de feriado em locais Off Premise: {weekly_data['holiday_off_premise'].sum():,}")

# Mostra as novas colunas no dataframe
print("\\nExemplo das novas colunas em uma semana de feriado (semana 3):")
display(weekly_data[weekly_data['week'] == 3].head())

Criando features de feriado com base na sua sugestão...
\nCriação das features de feriado concluída!
Semanas marcadas como feriado: 1,360,715
Semanas de feriado em locais On Premise: 183,862
Semanas de feriado em locais Off Premise: 1,170,572
\nExemplo das novas colunas em uma semana de feriado (semana 3):


,week,internal_store_id,internal_product_id,categoria,marca,premise,categoria_pdv,state,city,total_quantity,quantity_lag_1,quantity_lag_2,quantity_lag_3,quantity_lag_4,quantity_rolling_avg_2,quantity_rolling_avg_4,quantity_rolling_avg_8,is_holiday_week,holiday_on_premise,holiday_off_premise
158456,3,1001371918471115422,1120490062981954254,Package,Bud Light,Off Premise,Convenience,GA,Talking Rock,18.0,18.0,0.0,0.0,0.0,18.0,9.5,5.800000,1,0,1
158457,3,1001371918471115422,1394381856358939027,Package,Budweiser,Off Premise,Convenience,GA,Talking Rock,13.0,0.0,0.0,0.0,0.0,10.0,10.0,10.571429,1,0,1
158458,3,1001371918471115422,2626864159264988174,Package,Bud Ice,Off Premise,Convenience,GA,Talking Rock,2.0,0.0,0.0,0.0,0.0,1.0,4.0,5.000000,1,0,1
158459,3,1001371918471115422,4353552881410365573,Package,Natural Light,Off Premise,Convenience,GA,Talking Rock,4.0,7.0,0.0,0.0,0.0,7.0,4.0,2.200000,1,0,1
158460,3,1001371918471115422,4974205226846185245,Package,Natural Ice,Off Premise,Convenience,GA,Talking Rock,3.0,7.0,0.0,0.0,0.0,7.0,3.0,2.000000,1,0,1


In [45]:
# --- PASSO 7: CRIAR FEATURES DE FERIADOS ESTADUAIS ---

print("Criando features para feriados estaduais específicos (CA, CO, GA)...")
import numpy as np

# 1. Definir as semanas dos feriados únicos para cada estado, com base na nossa pesquisa
# (Converti as datas dos feriados para o número da semana em 2022)
ca_holiday_weeks = [13]  # Cesar Chavez Day
co_holiday_weeks = [13, 40] # Cesar Chavez Day, Frances Xavier Cabrini Day
ga_holiday_weeks = [15, 51] # Good Friday, State Holiday (Dec 23)

# 2. Criar a feature para feriados da Califórnia (CA)
weekly_data['is_holiday_ca'] = np.where(
    (weekly_data['state'] == 'CA') & (weekly_data['week'].isin(ca_holiday_weeks)), 1, 0
)

# 3. Criar a feature para feriados do Colorado (CO)
weekly_data['is_holiday_co'] = np.where(
    (weekly_data['state'] == 'CO') & (weekly_data['week'].isin(co_holiday_weeks)), 1, 0
)

# 4. Criar a feature para feriados da Geórgia (GA)
weekly_data['is_holiday_ga'] = np.where(
    (weekly_data['state'] == 'GA') & (weekly_data['week'].isin(ga_holiday_weeks)), 1, 0
)


# --- VERIFICAÇÃO ---
print("\\nCriação das features de feriados estaduais concluída!")

# Verificar se as colunas foram criadas e quantas marcações cada uma tem
ca_count = weekly_data['is_holiday_ca'].sum()
co_count = weekly_data['is_holiday_co'].sum()
ga_count = weekly_data['is_holiday_ga'].sum()

print(f"Semanas marcadas como feriado da Califórnia: {ca_count:,}")
print(f"Semanas marcadas como feriado do Colorado: {co_count:,}")
print(f"Semanas marcadas como feriado da Geórgia: {ga_count:,}")

# Se houver marcações, mostrar um exemplo
if ga_count > 0:
    print("\\nExemplo de feriado da Geórgia (semana 15 ou 51):")
    display(weekly_data[weekly_data['is_holiday_ga'] == 1].head())

Criando features para feriados estaduais específicos (CA, CO, GA)...
\nCriação das features de feriados estaduais concluída!
Semanas marcadas como feriado da Califórnia: 0
Semanas marcadas como feriado do Colorado: 122,948
Semanas marcadas como feriado da Geórgia: 88,852
\nExemplo de feriado da Geórgia (semana 15 ou 51):


,week,internal_store_id,internal_product_id,categoria,marca,premise,categoria_pdv,state,city,total_quantity,...,quantity_lag_4,quantity_rolling_avg_2,quantity_rolling_avg_4,quantity_rolling_avg_8,is_holiday_week,holiday_on_premise,holiday_off_premise,is_holiday_ca,is_holiday_co,is_holiday_ga
5915415,51,1001371918471115422,1009179103632945474,Package,Busch,Off Premise,Convenience,GA,Talking Rock,2.0,...,1.0,1.5,1.25,1.125000,0,0,0,0,0,1
5915416,51,1001371918471115422,1120490062981954254,Package,Bud Light,Off Premise,Convenience,GA,Talking Rock,10.0,...,10.0,12.5,11.25,14.500000,0,0,0,0,0,1
1356600,15,1001371918471115422,1394381856358939027,Package,Budweiser,Off Premise,Convenience,GA,Talking Rock,13.0,...,20.0,12.5,15.25,13.857143,0,0,0,0,0,1
5915417,51,1001371918471115422,1394381856358939027,Package,Budweiser,Off Premise,Convenience,GA,Talking Rock,13.0,...,9.0,19.5,14.50,13.250000,0,0,0,0,0,1
1356601,15,1001371918471115422,1454838625590783593,Package,Bud Ice,Off Premise,Convenience,GA,Talking Rock,3.0,...,0.0,0.0,19.50,12.500000,0,0,0,0,0,1


In [46]:
# --- DIAGNÓSTICO: POR QUE O FERIADO DA CALIFÓRNIA NÃO APARECE? ---

print("Investigando os dados da Califórnia e da semana 13...")
print("="*60)

# 1. Temos dados da Califórnia no geral?
ca_total_rows = len(weekly_data[weekly_data['state'] == 'CA'])
print(f"Número total de linhas de vendas semanais da Califórnia: {ca_total_rows:,}")

# 2. Temos dados da semana 13 no geral?
week_13_total_rows = len(weekly_data[weekly_data['week'] == 13])
print(f"Número total de linhas de vendas semanais da semana 13 (de todos os estados): {week_13_total_rows:,}")

# 3. A verificação final: Temos dados que são da CA E da semana 13 ao mesmo tempo?
combinacao_df = weekly_data[(weekly_data['state'] == 'CA') & (weekly_data['week'] == 13)]
num_combinacao = len(combinacao_df)
print(f"\\nNúmero de linhas que são da Califórnia E da semana 13: {num_combinacao}")

if num_combinacao == 0:
    print("\\nConclusão: Como suspeitado, não há registros de vendas em nosso dataset para a Califórnia durante a semana 13 de 2022.")
else:
    print("\\nConclusão: Encontramos dados para a CA na semana 13. Pode haver outro problema.")
    display(combinacao_df)

Investigando os dados da Califórnia e da semana 13...
Número total de linhas de vendas semanais da Califórnia: 432
Número total de linhas de vendas semanais da semana 13 (de todos os estados): 107,262
\nNúmero de linhas que são da Califórnia E da semana 13: 0
\nConclusão: Como suspeitado, não há registros de vendas em nosso dataset para a Califórnia durante a semana 13 de 2022.


In [ ]:
%pip install scikit-learn

A biblioteca 'scikit-learn' não foi encontrada. Instalando agora...
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 8.7/8.7 MB 62.3 MB/s  0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   --------------------- ------------------ 20.4/38.5 MB 97.6 MB/s eta 0:00:01
   ----------------------------------- ---- 34.6/38.5 MB 93.5 MB/s eta 0:00:01
   ---------------------------------------- 38.5/38.5 MB 62.4 MB/s  0:00:00

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------

In [49]:
# --- PASSO 8: PREPARAR DADOS PARA O MODELO ---
from sklearn.preprocessing import LabelEncoder

print("Iniciando a preparação dos dados para o treinamento...")

# 1. Lidar com colunas categóricas (texto)
# O LabelEncoder transforma cada categoria única em um número. Ex: 'CA' -> 0, 'CO' -> 1, 'GA' -> 2
categorical_features = [
    'categoria', 'marca', 'premise', 'categoria_pdv', 'state', 'city'
]

# Copiamos para não alterar o weekly_data original
data_for_training = weekly_data.copy()

for feature in categorical_features:
    encoder = LabelEncoder()
    # Cria uma nova coluna com o sufixo '_encoded' com os valores numéricos
    data_for_training[f'{feature}_encoded'] = encoder.fit_transform(data_for_training[feature])

print("Colunas de texto convertidas para números.")


# 2. Definir as 'features' (X) e o 'alvo' (y)
# O alvo (y) é o que queremos prever: 'total_quantity'
y = data_for_training['total_quantity']

# As features (X) são todas as pistas que preparamos
feature_columns = [
    'week',
    # Lags e Rolling Averages
    'quantity_lag_1', 'quantity_lag_2', 'quantity_lag_3', 'quantity_lag_4',
    'quantity_rolling_avg_2', 'quantity_rolling_avg_4', 'quantity_rolling_avg_8',
    # Features de Feriado (Federais e Estaduais)
    'is_holiday_week', 'holiday_on_premise', 'holiday_off_premise',
    'is_holiday_ca', 'is_holiday_co', 'is_holiday_ga',
    # Features Categóricas que acabamos de codificar
    'categoria_encoded', 'marca_encoded', 'premise_encoded',
    'categoria_pdv_encoded', 'state_encoded', 'city_encoded'
]
X = data_for_training[feature_columns]


# --- VERIFICAÇÃO ---
print("\\nDados prontos para o modelo!")
print(f"Nosso 'alvo' (y - o que queremos prever) tem {len(y):,} linhas.")
print(f"Nossas 'features' (X - as pistas) têm {X.shape[0]:,} linhas e {X.shape[1]} colunas.")
print("\\nExemplo das features (X) que o modelo vai usar:")
display(X.head())

Iniciando a preparação dos dados para o treinamento...
Colunas de texto convertidas para números.
\nDados prontos para o modelo!
Nosso 'alvo' (y - o que queremos prever) tem 6,140,206 linhas.
Nossas 'features' (X - as pistas) têm 6,140,206 linhas e 20 colunas.
\nExemplo das features (X) que o modelo vai usar:


,week,quantity_lag_1,quantity_lag_2,quantity_lag_3,quantity_lag_4,quantity_rolling_avg_2,quantity_rolling_avg_4,quantity_rolling_avg_8,is_holiday_week,holiday_on_premise,holiday_off_premise,is_holiday_ca,is_holiday_co,is_holiday_ga,categoria_encoded,marca_encoded,premise_encoded,categoria_pdv_encoded,state_encoded,city_encoded
433888,6,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0,0,0,0,4,1449,1,54,2,442
529002,7,1.0,0.0,0.0,0.0,1.0,1.000000,1.000000,0,0,0,0,0,0,4,1449,1,54,2,442
2042472,21,2.0,1.0,0.0,0.0,1.5,1.500000,1.500000,0,0,0,0,0,0,4,1449,1,54,2,442
2521528,25,1.0,2.0,1.0,0.0,1.5,1.333333,1.333333,1,1,0,0,0,0,4,1449,1,54,2,442
2891177,28,2.0,1.0,2.0,1.0,1.5,1.500000,1.500000,0,0,0,0,0,0,4,1449,1,54,2,442


In [50]:
# --- PASSO 9: DIVIDIR DADOS EM TREINO E VALIDAÇÃO ---
from sklearn.model_selection import train_test_split

print("Dividindo os dados em conjuntos de treino e validação (80/20)...")

# A função train_test_split faz a divisão de forma aleatória e estratificada
# test_size=0.2 significa que 20% dos dados irão para validação
# random_state=42 garante que a divisão seja sempre a mesma se rodarmos o código de novo
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# --- VERIFICAÇÃO ---
print("\\nDivisão concluída!")
print(f"Tamanho do conjunto de treino (X_train): {len(X_train):,} linhas")
print(f"Tamanho do conjunto de validação (X_val): {len(X_val):,} linhas")

Dividindo os dados em conjuntos de treino e validação (80/20)...
\nDivisão concluída!
Tamanho do conjunto de treino (X_train): 4,912,164 linhas
Tamanho do conjunto de validação (X_val): 1,228,042 linhas


In [51]:
# --- PASSO 10: TREINAR O MODELO CATBOOST ---

# 1. Instalar a biblioteca catboost, caso ainda não tenha
print("Instalando/Verificando a biblioteca 'catboost'...")
%pip install catboost -q

from catboost import CatBoostRegressor
print("\\nBiblioteca CatBoost pronta.")


# 2. Inicializar o modelo com alguns parâmetros
# iterations: O número máximo de "rodadas" de aprendizado.
# learning_rate: O quão rápido o modelo aprende.
# depth: A profundidade das "árvores de decisão" que o modelo constrói.
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=10,
    loss_function='RMSE', # Métrica de erro que o modelo tentará minimizar
    random_seed=42,
    verbose=100  # Mostra o progresso do treino a cada 100 rodadas
)

print("\\nIniciando o treinamento do modelo... (Isso pode levar alguns minutos)")

# 3. Treinar o modelo!
# Passamos os dados de treino (X_train, y_train) para ele aprender.
# Passamos os dados de validação (eval_set) para ele monitorar a própria performance e evitar "decoreba".
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50, # Para de treinar se a performance na validação não melhorar por 50 rodadas
    verbose=100
)

print("\\n--- TREINAMENTO CONCLUÍDO! ---")
print("Seu modelo de previsão de vendas foi treinado com sucesso.")

Instalando/Verificando a biblioteca 'catboost'...
Note: you may need to restart the kernel to use updated packages.
\nBiblioteca CatBoost pronta.
\nIniciando o treinamento do modelo... (Isso pode levar alguns minutos)
0:	learn: 85.9999628	test: 66.5104908	best: 66.5104908 (0)	total: 607ms	remaining: 10m 6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 62.60593379
bestIteration = 34

Shrink model to first 35 iterations.
\n--- TREINAMENTO CONCLUÍDO! ---
Seu modelo de previsão de vendas foi treinado com sucesso.


In [52]:
# --- PASSO 11: GERAR AS PREVISÕES PARA 2023 ---

print("Iniciando a geração das previsões para as 5 primeiras semanas de 2023...")

# 1. Pegar o último registro de cada combinação loja/produto de 2022
# Isso nos dará as informações mais recentes para basear nossas previsões
latest_records = data_for_training.loc[data_for_training.groupby(['internal_store_id', 'internal_product_id'])['week'].idxmax()]

# 2. Loop para prever cada uma das 5 semanas
all_predictions = []
current_features = latest_records.copy()

for week_num in range(1, 6):
    print(f"Prevendo a semana {week_num} de 2023...")
    
    # Atualizar a feature 'week' para a semana que queremos prever
    current_features['week'] = week_num
    
    # Garantir que as features estão na ordem correta que o modelo espera
    X_future = current_features[feature_columns]
    
    # Fazer a previsão!
    predictions = model.predict(X_future)
    
    # Garantir que não teremos previsões negativas
    predictions = np.maximum(0, predictions).round().astype(int)
    
    # Guardar as previsões em um formato amigável
    week_predictions = pd.DataFrame({
        'semana': week_num,
        'pdv': current_features['internal_store_id'],
        'produto': current_features['internal_product_id'],
        'quantidade_prevista': predictions
    })
    all_predictions.append(week_predictions)
    
    # ATUALIZAÇÃO PARA A PRÓXIMA SEMANA:
    # As previsões desta semana se tornam o 'lag_1' da próxima semana, e os outros lags são deslocados.
    # Esta é a parte mais complexa, que simula o tempo passando.
    current_features['quantity_lag_4'] = current_features['quantity_lag_3']
    current_features['quantity_lag_3'] = current_features['quantity_lag_2']
    current_features['quantity_lag_2'] = current_features['quantity_lag_1']
    current_features['quantity_lag_1'] = predictions

# 3. Juntar todas as previsões em um único dataframe
predictions_df = pd.concat(all_predictions, ignore_index=True)


# --- VERIFICAÇÃO ---
print("\\nPREVISÕES GERADAS COM SUCESSO!")
print(f"Total de previsões geradas: {len(predictions_df):,}")
print("Amostra do arquivo final de previsão:")
display(predictions_df.head(10))

# Salvar as previsões em um arquivo CSV
predictions_df.to_csv("previsoes_vendas_2023.csv", index=False, sep=';')
print("\\nArquivo 'previsoes_vendas_2023.csv' salvo com sucesso no seu diretório!")

Iniciando a geração das previsões para as 5 primeiras semanas de 2023...
Prevendo a semana 1 de 2023...
Prevendo a semana 2 de 2023...
Prevendo a semana 3 de 2023...
Prevendo a semana 4 de 2023...
Prevendo a semana 5 de 2023...
\nPREVISÕES GERADAS COM SUCESSO!
Total de previsões geradas: 5,114,685
Amostra do arquivo final de previsão:


,semana,pdv,produto,quantidade_prevista
0,1,1000237487041964405,1837429607327399565,3
1,1,1000237487041964405,4038588102284338370,4
2,1,1000237487041964405,5429216175252037173,3
3,1,1000237487041964405,596381974901127871,3
4,1,1000237487041964405,7270233133454638680,3
5,1,1000237487041964405,7370044109082767116,3
6,1,1000237487041964405,7405304019373961901,3
7,1,1000237487041964405,777251454728290683,3
8,1,1000237487041964405,8313805606242965556,3
9,1,1000275275922029725,1735457469340543861,32


\nArquivo 'previsoes_vendas_2023.csv' salvo com sucesso no seu diretório!


In [53]:
# --- PASSO 12: AVALIAÇÃO DETALHADA DO MODELO (O SEU "EVAL") ---
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

print("Calculando métricas de avaliação detalhadas no conjunto de validação...")

# 1. Usar o modelo para fazer previsões nos dados de validação que separamos
y_pred = model.predict(X_val)

# 2. Calcular as métricas de erro comparando o real (y_val) com o previsto (y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

# Calcular MAPE (Erro Percentual Absoluto Médio) com cuidado para não dividir por zero
epsilon = 1e-10 # Um número muito pequeno para evitar divisão por zero
mape = np.mean(np.abs((y_val - y_pred) / (y_val + epsilon))) * 100


# --- APRESENTAÇÃO DOS RESULTADOS (O SEU "EVAL") ---
print("\\n" + "="*40)
print("--- Relatório de Avaliação do Modelo ---")
print("="*40)

print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"  -> Interpretação: Em média, as previsões erraram em ~{rmse:.0f} unidades.")

print(f"\\nMAE (Mean Absolute Error): {mae:.2f}")
print(f"  -> Interpretação: Na média, o erro absoluto de cada previsão foi de {mae:.0f} unidades.")

print(f"\\nMAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"  -> Interpretação: Na média, as previsões tiveram um desvio de {mape:.1f}% em relação ao valor real.")

print(f"\\nR² (R-squared): {r2:.2f}")
print(f"  -> Interpretação: Nosso modelo consegue explicar {r2:.1%} da variação nas vendas.")
print("="*40)

Calculando métricas de avaliação detalhadas no conjunto de validação...
\n========================================
--- Relatório de Avaliação do Modelo ---
RMSE (Root Mean Squared Error): 62.61
  -> Interpretação: Em média, as previsões erraram em ~63 unidades.
\nMAE (Mean Absolute Error): 8.15
  -> Interpretação: Na média, o erro absoluto de cada previsão foi de 8 unidades.
\nMAPE (Mean Absolute Percentage Error): 202201.39%
  -> Interpretação: Na média, as previsões tiveram um desvio de 202201.4% em relação ao valor real.
\nR² (R-squared): 0.13
  -> Interpretação: Nosso modelo consegue explicar 12.5% da variação nas vendas.


In [54]:
# --- CÁLCULO DO WMAPE ---

print("Calculando o WMAPE (a métrica mais importante para este problema)...")

# Relembrando, y_val são as vendas reais e y_pred são as previsões do modelo para o conjunto de validação.

# A fórmula é: Soma dos erros absolutos / Soma dos valores reais
wmape = np.sum(np.abs(y_val - y_pred)) / np.sum(np.abs(y_val)) * 100


# --- RESULTADO ---
print("\\n" + "="*55)
print("--- Métrica WMAPE (Weighted Mean Absolute Percentage Error) ---")
print("="*55)
print(f"WMAPE: {wmape:.2f}%")
print("\\n  -> Interpretação: O erro total do nosso modelo corresponde a " \
      f"{wmape:.1f}% do volume total de vendas no período de validação.")
print("="*55)

Calculando o WMAPE (a métrica mais importante para este problema)...
\n=======================================================
--- Métrica WMAPE (Weighted Mean Absolute Percentage Error) ---
WMAPE: 94.22%
\n  -> Interpretação: O erro total do nosso modelo corresponde a 94.2% do volume total de vendas no período de validação.


In [55]:
# --- PASSO 9 e 10 (VERSÃO CORRIGIDA): DIVISÃO TEMPORAL E RETREINAMENTO ---

print("Refazendo o processo com uma divisão de dados baseada no tempo...")

# 1. Ordenar os dados por semana
data_for_training = data_for_training.sort_values(by='week')

# 2. Fazer a divisão temporal
# Vamos usar as semanas até a semana 42 para treino, e o resto para validação.
split_week = 42 
train_data = data_for_training[data_for_training['week'] <= split_week]
val_data = data_for_training[data_for_training['week'] > split_week]

# Separar features (X) e alvo (y) para cada conjunto
X_train = train_data[feature_columns]
y_train = train_data['total_quantity']
X_val = val_data[feature_columns]
y_val = val_data['total_quantity']

print(f"Divisão temporal concluída: {len(X_train):,} linhas de treino, {len(X_val):,} linhas de validação.")


# 3. Inicializar um novo modelo com parâmetros mais seguros
print("\\nInicializando um novo modelo com parâmetros mais seguros...")
model_v2 = CatBoostRegressor(
    iterations=2000, # Aumentamos as iterações pois esperamos que demore mais para convergir
    learning_rate=0.05,
    depth=7, # Profundidade menor para evitar overfitting
    loss_function='RMSE',
    random_seed=42,
    verbose=200
)

# 4. Retreinar o modelo com os dados corretamente divididos
print("Iniciando o retreinamento...")
model_v2.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=100 # Aumentamos a paciência do early stopping
)


# 5. Reavaliar o WMAPE
print("\\n--- Reavaliando o Modelo v2 ---")
y_pred_v2 = model_v2.predict(X_val)

wmape_v2 = np.sum(np.abs(y_val - y_pred_v2)) / np.sum(np.abs(y_val)) * 100

print(f"Novo WMAPE: {wmape_v2:.2f}%")

Refazendo o processo com uma divisão de dados baseada no tempo...
Divisão temporal concluída: 4,952,155 linhas de treino, 1,188,051 linhas de validação.
\nInicializando um novo modelo com parâmetros mais seguros...
Iniciando o retreinamento...
0:	learn: 91.8266380	test: 14.8056012	best: 14.8056012 (0)	total: 419ms	remaining: 13m 57s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 13.62186428
bestIteration = 32

Shrink model to first 33 iterations.
\n--- Reavaliando o Modelo v2 ---
Novo WMAPE: 94.34%


In [56]:
# --- INVESTIGANDO O CONJUNTO DE VALIDAÇÃO (y_val) ---

print("Investigando o conjunto de validação (y_val) para entender o WMAPE...")
print(f"Número total de linhas no y_val: {len(y_val):,}")

# Contar quantas vendas são zero no conjunto de validação
num_vendas_zero = (y_val == 0).sum()
print(f"Número de vendas zero no y_val: {num_vendas_zero:,}")

# Calcular a porcentagem de vendas zero
porcentagem_vendas_zero = (num_vendas_zero / len(y_val)) * 100
print(f"Porcentagem de vendas zero no y_val: {porcentagem_vendas_zero:.2f}%")

# Calcular a soma total das vendas no y_val (o denominador do WMAPE)
soma_total_vendas = y_val.sum()
print(f"Soma total das vendas no y_val (denominador do WMAPE): {soma_total_vendas:,.2f}")

# Mostrar algumas estatísticas de y_val
print("\\nEstatísticas descritivas de y_val:")
print(y_val.describe())

Investigando o conjunto de validação (y_val) para entender o WMAPE...
Número total de linhas no y_val: 1,188,051
Número de vendas zero no y_val: 0
Porcentagem de vendas zero no y_val: 0.00%
Soma total das vendas no y_val (denominador do WMAPE): 6,004,308.94
\nEstatísticas descritivas de y_val:
count    1.188051e+06
mean     5.053915e+00
std      1.426702e+01
min      4.166700e-02
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      2.472000e+03
Name: total_quantity, dtype: float64
